### Este es el ejemplo 4.3. Problema del Apostador (Gambler’s Problem) del libro de Sutton. 

Un apostador tiene la oportunidad de hacer apuestas a los resultados de una secuencia de tiros de una moneda. 
Si la moneda cae cara, gana tantos dólares como apostó en esa tirada.
Si la moneda cae ceca, pierde lo apostado. El juego termina cuando un apostador gana alcanzando su objetivo de $100, o pierde quedándose sin dinero.

En cada tirada el apostador debe decidir qué porción de su capital quiere apostar, una cantidad entera de dólares.
El problema puede ser formulado como un MDP finito sin descuento, episódico.

Los estados posibles del capital del apostador son: s ∈ {1, 2, . . . , 99}.

La acciones son apuestas,  a ∈ {0, 1, . . . , min(s, 100 − s)}. 

La recompensa es cero en todas las transiciones excepto en aquellas en que el apostador alcanza su objetivo, en que la recompensa es +1.

La función de estado-valor da la probabilidad de ganar desde cada estado. Una política es una función de niveles de capital a apuestas. La política óptima maximiza la probabilidad de obtener el objetivo. Llamemos p_h la probabilidad de que una moneda salga cara. Si p_h es conocida, entonces el problema se puede resolver, por ejemplo, con iteración de valor.

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
if "../" not in sys.path:
  sys.path.append("../") 


### Exercise

Implementar iteración de valor para el problema del apostador y resolverlo para p_h = 0.25 y p_h = 0.55.


In [2]:
def one_step_lookahead(s, V, rewards):
        """
        Función auxiliar que calcula el valor de todas las acciones dado un estado.
        
        Args:
            s: El capital del apostador. Entero.
            V: El vector que contiene los valores en cada estado.
            rewards: El vector recompensa.
                        
        Returns:
            Un vector que contiene el valor esperado de cada acción.
            Su longitud es igual a la cantidad de acciones.
        """
        
        # Implementar!
        
        return A

In [15]:
def value_iteration_for_gamblers(p_h, theta_eval=1e-10, discount_factor=1.0, debug=0,
                                 one_step_lookahead_fn=one_step_lookahead, max_steps_eval=0, max_steps_opt=0):
    """
    Args:
        p_h: Probabilidad de que una moneda caiga cara
        theta_eval: theta para la evaluacion, diferencia entre V_k(s) y V_k+1(s)
        theta_opt: theta para la optimizacion, diferencia relativa entre la mejor accion y la mas probable.
        discount_factor:
        debug:  {0, no debuggea nada.
                 1, debug parcial.
                 2, debug total}
        one_step_lookahead_fn: funcion encargada de obtener el valor esperado de cada accion.
        max_steps_eval: maxima cantidad de pasos a iterar en cada evaluacion.
        max_steps_opt: maxima cantidad de pasos a iterar en la optimizacion de politica.
    """
     
    # constantes
    n_S = 101
    n_A = 51
    
    # definimos una politica random para iniciar
    policy = np.zeros([n_S, n_A])
    
    for s in range(51):
        for a in range(0, s+1):
            policy[s][a] = 1/(s+1)

    for s in range(51,101):
        n_a_posibles = min(s,100-s)+1
        for a in range(n_a_posibles):
            policy[s][a] = 1/(n_a_posibles)
         
    # definimos nuestro env a mano
    env = {}

    for s in range(0, 101):
        env[s] = {}

        for a in range(min(s,100-s)+1):        
            env[s][a] = []

            if (a==0):
                r = 1.0 if (s==100) else 0.0
                done = True if ((s==0) or (s==100)) else False
                env[s][a] = [(1.0, s, r, done)]
            else:
                sig_estado_si_gano = s+a
                sig_estado_si_pierdo = s-a
                done_si_gano = True if (sig_estado_si_gano==100) else False
                done_si_pierdo = True if (sig_estado_si_pierdo==0) else False
                r = 1.0 if (sig_estado_si_gano==100) else 0.0
                env[s][a] = [(p_h, sig_estado_si_gano, r, done_si_gano), ((1-p_h), (s-a), 0.0, done_si_pierdo)]
    
    
    # inicio del algoritmo iterativo de optimizacion
    paso = 1

    while True:
        if (debug):
            if (paso!=1):
                print(f"\nINICIO DEL PASO {paso}")
            else:
                print(f"INICIO DEL PASO {paso}")
        
        # policy evaluation
        pasos_eval = 0
        V = np.zeros(n_S)

        while True:

            delta = 0
            for s in range(1, n_S-1):
                v = 0
                for a in env[s]:
                    for  prob, next_state, reward, done in env[s][a]:
                        v += policy[s][a] * prob * (reward + discount_factor * V[next_state])
                delta = max(delta, np.abs(v - V[s]))

                # for debug:
                #print(f"Paso {pasos+1} | Estado {s} | V_previo = {V[s]}, V_nuevo = {v}")

                V[s] = v  
            pasos_eval += 1

            if delta < theta_eval:
                termino_correctamente = True
                break

            if (max_steps_eval):
                if (pasos_eval == max_steps_eval):
                    termino_correctamente = False
                    break

        if (debug):
            if (termino_correctamente):
                print(f"La eval. en el paso {paso} terminó en {pasos_eval} pasos.")
            else:
                print(f"En el paso {paso} se alcanzaron los pasos maximos en la eval.")


        # en este paso, ya tenemos nuestro V actualizado tras la evaluacion

        politica_estable = True

        for s in range(1, n_S-1):

            a_mas_probable = policy[s].tolist().index(max(policy[s]))
            # OJO, hay que pensar una manera de que cuando las p sean todas iguales, la cambie por 1.
            #print(f"state = {s}, a mas probable = {a_mas_probable}")

            acciones = []        
            for a in env[s]:
                retorno_a = 0
                for transicion in env[s][a]:
                    p_transicion = transicion[0]
                    proximo_estado = transicion[1]
                    reward = transicion[2]
                    retorno_a += p_transicion*(reward + discount_factor*V[proximo_estado])
                acciones.append((a, retorno_a))


            mejor_a = max(acciones, key=lambda x: x[1])[0]

            #diferencia = acciones[mejor_a][1] - acciones[a_mas_probable][1]
            #diferencia_rel = (diferencia)/(acciones[a_mas_probable][1])
            diferencia_rel = (acciones[mejor_a][1])/(acciones[a_mas_probable][1])

            if (debug == 3):
                print(f"paso {paso} | s = {s} | a_priori = {a_mas_probable} | dsp = {mejor_a} | modifica= {diferencia_rel > 1}")
                print(f"a_priori p = {acciones[a_mas_probable][1]} | dsp p = {acciones[mejor_a][1]} | dif rel = {diferencia_rel -1}")
                #print(acciones)
                
            if ((diferencia_rel > 1) and (mejor_a != a_mas_probable)):
                if (debug==2):
                    print(f"paso {paso} | s = {s} | a_priori = {a_mas_probable} | dsp = {mejor_a} | dif = {diferencia_rel-1}")
                    #print(f"a_priori p = {acciones[a_mas_probable][1]} | dsp p = {acciones[mejor_a][1]}")

                politica_estable = False
                pol_s = np.zeros(51)
                pol_s[mejor_a] = 1
                policy[s] = pol_s


        if (politica_estable):
            print(f"Se encontro una politica optima tras {paso} paso/s.\n")
            break

        if (max_steps_opt):
            if (paso == max_steps_opt):
                print(f"Se corta la optimizacion de la politica al llegar a los pasos maximos ({max_steps_opt}).\n")
                break

        paso += 1
    
    
    return policy, V

In [19]:
policy, v = value_iteration_for_gamblers(p_h=0.25, theta_eval=1e-10, debug=0, max_steps_opt=20)

print("Política optimizada:")
print(policy)
print("")

print("Función de valor óptima:")
print(v)
print("")

Se encontro una politica optima tras 5 paso/s.

Política optimizada:
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]

Función de valor óptima:
[0.00000000e+00 7.28611644e-05 2.91444667e-04 6.95264566e-04
 1.16577869e-03 1.77125505e-03 2.78105828e-03 4.03661210e-03
 4.66311477e-03 5.60141644e-03 7.08502024e-03 9.04088770e-03
 1.11242331e-02 1.56796459e-02 1.61464484e-02 1.69534413e-02
 1.86524591e-02 1.98260623e-02 2.24056658e-02 2.73847344e-02
 2.83400810e-02 3.04945467e-02 3.61635508e-02 3.84959101e-02
 4.44969325e-02 6.25000000e-02 6.27185835e-02 6.33743340e-02
 6.45857937e-02 6.59973361e-02 6.78137652e-02 7.08431749e-02
 7.46098363e-02 7.64893443e-02 7.93042493e-02 8.37550607e-02
 8.96226631e-02 9.58726994e-02 1.09538938e-01 1.10939345e-01
 1.13360324e-01 1.18457377e-01 1.21978187e-01 1.29716997e-01
 1.44654203e-01 1.47520243e-01 1.53983640e-01 1.70990652e-01
 1.77987730e-01 1

In [20]:
policy, v = value_iteration_for_gamblers(p_h=0.55, theta_eval=1e-10, debug=0, max_steps_opt=20)

print("Política optimizada:")
print(policy)
print("")

print("Función de valor óptima:")
print(v)
print("")

Se encontro una politica optima tras 9 paso/s.

Política optimizada:
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]

Función de valor óptima:
[0.         0.18181818 0.33057851 0.45229151 0.55187487 0.63335216
 0.7000154  0.75455806 0.79918386 0.83569589 0.86556936 0.89001129
 0.91000924 0.92637119 0.93975825 0.95071129 0.95967288 0.96700508
 0.97300416 0.97791249 0.9819284  0.98521415 0.98790248 0.99010203
 0.99190166 0.99337409 0.9945788  0.99556447 0.99637093 0.99703076
 0.99757062 0.99801233 0.99837372 0.99866941 0.99891134 0.99910927
 0.99927122 0.99940373 0.99951214 0.99960084 0.99967342 0.9997328
 0.99978138 0.99982113 0.99985365 0.99988026 0.99990203 0.99991984
 0.99993442 0.99994634 0.9999561  0.99996408 0.99997061 0.99997596
 0.99998033 0.9999839  0.99998683 0.99998923 0.99999119 0.99999279
 0.9999941  0.99999517 0.99999605 0.99999677 0.99999736 0.99999784
 0.99999823 0

In [5]:
# Plotear la política final (apuesta) vs estado (capital)

# Implementar!

In [6]:
# Plotear capital vs política final

# Implementar!
